In [1]:
import pandas as pd
import sys
sys.path.append('/home/bupi/Documents/pdy/hs/hsle/src')
import HsleCandidateGenerationUtils as hsle
import itertools
from tqdm import tqdm

In [2]:
fn = '/home/bupi/Documents/pdy/hs/hsle/data/exportcomments-outputs/20200323_20200325/processed/merged.csv'
df = pd.read_csv(fn)
df['name'] = hsle.uniNorm(df['Name (click to view profile)'].astype(str))
print(df.shape)
display(df.head(2))

(58725, 16)


,cId,nId,Name (click to view profile),Profile ID,Date,Likes,Comment,(view source),postId,nChars,MsgUni,atleast1MM,MsgUniSeg,LexFound,PostURL,name
0,1,0,Nunu,ID: 100004238449395,2020-03-24 13:01:38,1,အမေစုသက်ရှည်ကျန်းမာပါစေ🙏,view comment,25032020_0072,24,အမေစုသက်ရှည်ကျန်းမာပါစေ🙏,True,အမေ စု သက် ရှည် ကျန်းမာ ပါစေ 🙏,NaN,https://www.facebook.com/7daynews/posts/315261...,Nunu
1,3,0,Nyi Nyi,ID: 100046335742147,2020-03-24 13:02:29,2,Educating people is essential.. Teach people t...,view comment,25032020_0072,289,Educating people is essential.. Teach people t...,False,Educatingpeopleisessential..Teachpeopletodofre...,NaN,https://www.facebook.com/7daynews/posts/315261...,Nyi Nyi


## Grab DF

In [4]:
activity_df = df[['postId','cId','nId','name','Profile ID', 'LexFound']]
activity_df.columns = ['postid','cid','nid','name','profile_id', 'lex_found']
activity_df['profile_id'] = activity_df.profile_id.apply(lambda x:x.split()[-1])
activity_df.head(2)

,postid,cid,nid,name,profile_id,lex_found
0,25032020_0072,1,0,Nunu,100004238449395,NaN
1,25032020_0072,3,0,Nyi Nyi,100046335742147,NaN


## Accounts

In [6]:
accounts_df = activity_df[['name','profile_id']].drop_duplicates(['profile_id'])
print(activity_df.shape)
print(accounts_df.shape)
accounts_df = accounts_df[['profile_id','name']]
display(accounts_df.head(2))
accounts_df.to_csv('../results/networks/20200403_20200405_accounts.csv', sep='|', index=False)

(58725, 6)
(44114, 2)


,profile_id,name
0,100004238449395,Nunu
1,100046335742147,Nyi Nyi


## Connections
### Link by Comment Thread

Accounts that comment under the same comment are considered connected.

In [8]:
activity_df['id_name'] = [i+'|'+n for i,n in zip(activity_df.profile_id, activity_df.name)]
common_thread_link = activity_df.groupby(
    ['postid','cid'])['id_name'].apply(lambda x: ','.join(x))

58725it [00:00, 597439.35it/s]


In [9]:
source, target = [], []
for cohort in tqdm(common_thread_link.values):
    for s, t in itertools.combinations(cohort.split(','), 2):
        source.append(s)
        target.append(t)

100%|██████████| 49243/49243 [00:00<00:00, 340923.16it/s]


In [10]:
comment_thread_edges = pd.DataFrame({
    'source': source,
    'target': target
})
comment_thread_edges.to_csv(
    '../results/networks/20200403_20200405_comment_thread_edges.csv', index=False)
comment_thread_edges.head(2)

,source,target
0,100049298942225|Thway Thit,100010925860347|Nay Myo
1,100049298942225|Thway Thit,100017952098805|Shwe Mar


In [11]:
comment_thread_edges.groupby('source')['target'].count().sort_values(ascending=False)

source
100010233370110|Yan Naing Win      457
100030055881958|Thar Shwe Toke     418
100014670192351|Thura Ye Yint      273
100013695460874|Saw Soe Moe Han    264
100026402544012|Thaw Zin           240
                                  ... 
100017962271588|Yee Yee              1
100017954734886|Hnin Nu Nu Wai       1
100017836024246|Nwe Nwe              1
100017796042457|Aung Soe Minn        1
100018206552433|Mu Ngwe Winn         1
Name: target, Length: 6654, dtype: int64